<a href="https://colab.research.google.com/github/nyanta012/chatgpt_api_practice/blob/main/section9/section9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colaboratoryのショートカットキー
1. **現在のセルを実行し、次のセルを選択**: `Shift + Enter`
2. **コードセルをテキストセルにする**: `Ctrl + M M`
3. **コードセルを上に追加**: `Ctrl + M A`
4. **コードセルを下に追加**: `Ctrl + M B`
5. **セルの削除**: `Ctrl + M D`

# 演習

FunctionCallingを用いてユーザーからの入力を基に、2つの参照情報を切り替える実装をしてみよう

# API KEYの設定など

In [1]:
%%capture
!pip install langchain==0.0.340
!pip install llama-index==0.9.7
!wget https://github.com/nyanta012/chatgpt_api_practice/raw/main/pdf/コンプライアンスのすべて.pdf
!wget https://github.com/nyanta012/chatgpt_api_practice/raw/main/pdf/健康のすべて.pdf

In [2]:
import getpass
import json
import os
from pathlib import Path

from openai import OpenAI
from IPython.display import Markdown, display
from langchain.chat_models import ChatOpenAI
from llama_index import (GPTVectorStoreIndex, LLMPredictor, ServiceContext,
                         StorageContext, VectorStoreIndex, download_loader,
                         load_index_from_storage, PromptTemplate)

apikey = getpass.getpass(prompt="OpenAIのAPIキーを入力してください")
os.environ["OPENAI_API_KEY"] = apikey # llama indexは環境変数じゃないとエラーがでる？

client = OpenAI(api_key=apikey)

OpenAIのAPIキーを入力してください··········


# 実装してみよう

# 実装例(一例)

In [3]:
# llama indexのpdf readerを使う
PDFReader = download_loader("PDFReader")
loader = PDFReader()

# テキスト情報を抽出する
document_1 = loader.load_data(file=Path("コンプライアンスのすべて.pdf"))
document_2 = loader.load_data(file=Path("健康のすべて.pdf"))

# テキストをインデックス化する
INDEX_1 = VectorStoreIndex.from_documents(document_1)
INDEX_2 = VectorStoreIndex.from_documents(document_2)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
def get_index_engine(index_name: str):
    """
    プロンプトを回答するのに適切なindexを提供する
    """
    TEMPLATE = (
        "下記の情報が与えられています。 \n"
        "---------------------\n"
        "{context_str}"
        "\n---------------------\n"
        "この情報を参照して次の質問に答えてください:  {query_str}\n"
    )
    PROMPT = PromptTemplate(TEMPLATE)

    if index_name == "コンプライアンス":
        return INDEX_1.as_query_engine(text_qa_template=PROMPT)
    elif index_name == "健康問題":
        return INDEX_2.as_query_engine(text_qa_template=PROMPT)
    else:
        return None

In [5]:
def get_chatgpt_response_from_different_sources(user_input: str) -> None:
    # 与えた関数の説明を記載
    my_functions = [
        {
            "name": "get_index_engine",
            "description": "ユーザーの質問・指示に答えるために使われる関数",
            "parameters": {
                "type": "object",
                "properties": {
                    "index_name": {
                        "type": "string",
                        "enum": ["コンプライアンス", "健康問題", "該当なし"],
                        "description": "ユーザーの質問に答えるために、関連が高い単語を1つ選択してください。",
                    },
                },
                "required": ["index_name"],
            },
        }
    ]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": user_input}],
        functions=my_functions,
        function_call={"name": "get_index_engine"},
    )

    message = response.choices[0].message
    function_args = json.loads(message.function_call.arguments)

    engine = get_index_engine(function_args["index_name"])

    if engine:
        function_response = engine.query(user_input)
        source = function_response.get_formatted_sources(200)  # 200文字まで表示
        used_index = function_args["index_name"]
        display(
            Markdown(
                f"## ユーザー入力\n\n{user_input}\n\n## AI回答\n\n{function_response}\n\n## 参照情報({used_index}) \n\n{source}"
            )
        )
    else:
        print("関連する情報がありません")

In [6]:
get_chatgpt_response_from_different_sources("AIとコンプライアンスについて教えてください")

## ユーザー入力

AIとコンプライアンスについて教えてください

## AI回答

AIとコンプライアンスについては、以下のような情報が与えられています。

- 36. コンプライアンスと人工知能（ AI）: この章では、人工知能（AI）の発展により、コンプライアンス活動における自動化や洞察力の向上が期待されています。AIによるリスク評価と予測、自動監視とデータ分析、AIの倫理と透明性などについて詳しく解説されています。また、AIの活用に伴う課題やリスク、人間とAIの役割分担に関する考察も取り上げられています。

この情報からは、AIがコンプライアンス活動において重要な役割を果たすことが示唆されています。AIはリスク評価や予測、自動監視、データ分析などの作業を効率化し、洞察力やリスク管理能力を向上させることが期待されています。また、AIの活用には課題やリスクも存在し、人間とAIの役割分担についても考慮する必要があります。

## 参照情報(コンプライアンス) 

> Source (Doc id: b948a45f-7dba-4aca-a5cb-ef6ffaec4b3a): 【35. コンプライアンスとクラウドコンピューティング】 （ 500文字程度）  
クラウドコンピューティングの普及により、データ保護とコンプライアンスの課題が浮上
しています。 この章では、 コンプライアンスとクラウドコンピューティングの関連性につい
て詳しく解説します。 データの保護とプライバシー確保、 クラウドプロバイダとの契約と監
査、データの移転と適法性の確保などに関する具体的なア...

> Source (Doc id: 8c025272-4529-4937-94f6-99dbc6de2130): テクノロジーの活用とコンプライアンス】 （ 500文字程度）  
テクノロジーの進化は、コンプライアンス活動において革新的な手法とチャンスをもたら
しています。 この章では、 テクノロジーの活用とコンプライアンスの関連性について探求し
ます。 AI（人工知能）や機械学習、ブロックチェーンなどのテクノロジーがコンプライアン
スプロセスやリスク管理にどのように活用されているかを解説します。 また...

In [7]:
get_chatgpt_response_from_different_sources("AIと健康について教えてください")

## ユーザー入力

AIと健康について教えてください

## AI回答

AI（人工知能）は、健康に関連するさまざまな分野で活用されています。予防医学の一環として、AIは健康リスクの評価や予測、疾病の早期発見、治療計画の立案などに役立てられています。AIは大量のデータを解析し、パターンや傾向を見つけ出すことが得意であり、これによって医師や研究者はより正確な診断や治療を行うことができます。

また、AIは健康管理や予防にも活用されています。モバイルアプリやウェアラブルデバイスと連携し、個人の健康データを収集・分析し、健康状態や生活習慣の改善点を提案することがあります。さらに、AIは健康情報の提供や相談にも活用されており、オンラインで医療相談を行ったり、症状に基づいて適切な医療機関を紹介したりすることがあります。

AIの活用によって、健康管理や予防医学の効率性や精度が向上し、個々の健康状態の把握や疾病の予防・治療に役立てることができます。しかし、AIの活用には注意が必要であり、個人情報の保護や医療専門家の判断とのバランスなど、さまざまな課題も存在します。

## 参照情報(健康問題) 

> Source (Doc id: 373c56ba-cb8a-45df-97a9-ce07c59395c6): 健康を維持するためには、疾病の予防が重要です。予防医学の手法としては、予防接種やス
クリーニングテストの実施、健康診断の受診などがあります。また、リスクファクターの管
理や健康への適切なアドバイスも予防医学の重要な要素です。自身の健康リスクを正しく
評価し、 予防医学の 原則に基づいた行動を取ることで、 病気や健康問題を予防することが可
能となります。  
 
１４．健康への環境への配慮  ...

> Source (Doc id: f191de28-6404-498c-afb8-e6926d021bd0): 健康に関する新しい知識や最新の研究
結果を学び続けることで、自身の健康管理や健康への取り組みを進化させることができま
す。生涯学習の一環として、健康に関するセミナーやワークショップへの参加、専門書やオ
ンラインの健康情報の活用、健康に関連するコミュニ ティや団体への参加などが挙げられ
ます。 自身の健康についての知識を深めることで、 より良い健康状態を維持するための選択
や行動が可能となりま...

In [8]:
get_chatgpt_response_from_different_sources("にゃんたについて教えてください")

関連する情報がありません
